In [1]:
from fuzzywuzzy import fuzz
import station_geo
import pandas as pd
import pickle

In [2]:
pd.set_option('display.max_columns', None)
df1 = pd.read_csv('byStation.csv')
df1.columns = ['STATION','DATE','ENTRIES','EXITS']
df2 = pd.read_csv('data/DOITT_SUBWAY_STATION_01_13SEPT2010.csv')
df2.columns = ['URL','OBJECTID','NAME','the_geom','LINE','NOTES']

In [3]:
df1_stations = list(df1.STATION.unique())
df2_stations = list(df2.NAME.unique())
df2_geo = list(df2.the_geom.unique())
d_2_geo = [tuple(map(float, (str(x)[7:-1].split())))[::-1] for x in df2_geo]
df2_stat_geo = list(zip(df2_stations, d_2_geo))

In [4]:
df2_stat_geo

[('Astor Pl', (40.73005400028978, -73.99106999861966)),
 ('Canal St', (40.71880300107709, -74.00019299927328)),
 ('50th St', (40.76172799961419, -73.98384899986625)),
 ('Bergen St', (40.68086213682956, -73.97499915116808)),
 ('Pennsylvania Ave', (40.66471445143568, -73.89488591154061)),
 ('238th St', (40.88466700064975, -73.90087000018522)),
 ('Cathedral Pkwy (110th St)', (40.800581558114956, -73.95806670661364)),
 ('Kingston - Throop Aves', (40.67991899941601, -73.94085899871263)),
 ('65th St', (40.74971952935675, -73.8987883783301)),
 ('36th St', (40.75196004401078, -73.92901818461539)),
 ('Delancey St - Essex St', (40.71830605618619, -73.98740940202974)),
 ('Van Siclen Ave', (40.67802821447783, -73.89165772702445)),
 ('Norwood Ave', (40.68152000045683, -73.87962599910783)),
 ('104th-102nd Sts', (40.69516599823373, -73.84443500029684)),
 ('DeKalb Ave', (40.690648119969794, -73.98177094440949)),
 ('Beach 105th St', (40.58326843810286, -73.82758075034528)),
 ('Beach 90th St', (40.58809

In [5]:
def matcher(stations_1, stations_2):
    d = {}
    for x in df1_stations:
        d[x] = []
        for y in df2_stat_geo:
            d[x].append((y[0], fuzz.ratio(x,y), y[1]))
    return {i[0] : max(i[1], key=lambda x: x[1]) for i in d.items()}

In [6]:
matched = matcher(df1_stations, df2_stations)

In [7]:
df1['name'] = [matched[x][0] for x in df1['STATION']]

In [8]:
df1['score'] = [matched[x][1] for x in df1['STATION']]

In [9]:
df1['geo'] = [matched[x][2] for x in df1['STATION']]

In [10]:
#df1['geo'] = [df2[df2["NAME"] == x].the_geom for x in df1['name']]

In [11]:
#df2[df2["NAME"] == '1st Ave']['the_geom']

In [12]:
#df_backup['geo'] = [matched[x][0] for x in df['STATION']]

In [13]:
df1.head()

,STATION,DATE,ENTRIES,EXITS,name,score,geo
0,1 AV,01/01/2018,8427,9924,1st Ave,11,"(40.6508606878022, -73.94945514035334)"
1,1 AV,01/02/2018,18668,21328,1st Ave,11,"(40.6508606878022, -73.94945514035334)"
2,1 AV,01/03/2018,20417,22357,1st Ave,11,"(40.6508606878022, -73.94945514035334)"
3,1 AV,01/04/2018,12010,13472,1st Ave,11,"(40.6508606878022, -73.94945514035334)"
4,1 AV,01/05/2018,18891,21966,1st Ave,11,"(40.6508606878022, -73.94945514035334)"


In [14]:
df_final = df1

In [15]:
df_final.to_pickle('df_final.pickle')